# 引言

本项目为帮助初学者入门profile工作所编写，是本组针对主流的profile工具所设计和编写的，包括一些小型模型以测试对于profiling的理解。如有不周到之处请谅解，我们也将不断完善该文档。

# 1. NVIDIA Nsight Compute

## 1.1 介绍

您如果需要安装NVIDIA Nsight，可以在https://developer.nvidia.com/tools-overview/nsight-compute/get-started，网址上获取安装包进行安装，安装流程省略。Nsight Compute 工具将其测量库插入到应用程序进程中，这允许分析器拦截与CUDA用户模式驱动程序的通信。此外，当检测到内核启动时，库可以从GPU收集所请求的性能指标。然后将结果传输回前端。Nsight Compute主要是针对CUDA Kernel较少的程序设计的，适合小型程序的profiling工作。

## 1.2 ncu常用指令

最直接且常用的用法是下面两个命令，如想了解详细解释，可见各细节部分<br>
ncu --set full -o profile_result ./application<br>
ncu --set default -o profile_result python your_script.py

### 1.2.1 基础性能分析

<b>ncu --set full -o profile_result ./application</b> <br>
<br>
<b>--set参数</b>：<br>default（基础性能概览：包含内核启动统计、占用率和GPU吞吐量等约35个核心指标，分析开销较小，适合初步性能评估）<br>detailed（深入性能分析：在default基础上增加了指令统计、内存工作负载分析和Warp状态统计等，提供约157个更详细的指标，用于深入探究性能瓶颈根源）<br>full（全面数据收集：提供最全面的指标（约162个），包含更详尽的内存工作负载图表和数据表。对系统开销最大，通常用于深度优化）<br>source（源代码关联分析：专注于约47个与源代码映射相关的性能计数器，需要编译时使用-lineinfo等选项生成调试信息，便于进行指令级优化）<br>
<br>
<b>-o参数</b>：profile_result为输出报告文件的名字 <br>
<br>
<b>./application</b>为需要执行的程序 <br>
<br>
完成后会在服务器上产生一个名称为profile_result.ncu-rep的文件，可以在本地用NVIDIA Nsight Compute打开 <br>

### 1.2.2 指标收集

<b>ncu --metrics dram__bytes.sum.per_second -o profile_result ./application</b><br>
<br>
<b>--metrics参数</b>：用以收集特定的指标，可以使用逗号分隔不同指标或使用正则表达式<br>
<br>
可以使用ncu --query-metrics命令查询所有可用的指标。<br>
<br>
常用的指标有：<br>
sm__cycles_active.avg.pct_of_peak_sustained_elapsed，SM活跃周期占比<br>
sm__warps_active.avg.pct，活跃Warp百分比<br>
dram__bytes.sum.per_second，显存带宽使用率<br>
smsp__sass_thread_inst_executed_op_fadd_pred_on.sum，已执行的特定类型指令（如浮点加法）数量<br>
Occupancy，理论占用率与实际占用率差距过大常表明工作负载不均衡<br>
l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum，全局内存加载请求数<br>
l1tex__t_sectors_pipe_lsu_mem_global_op_st.sum，全局内存存储请求数，分析缓存局部性<br>
<br>
第一行的命令完成后即收集完成内存带宽指标，结果输出到指定文件当中

### 1.2.3 结果过滤与聚焦

<b>ncu -k "my_kernel" -o profile_result ./application</b><br>
<br>
“my_kernel”处指定需要详细分析的内核函数，如何获得内核名称？<br>
直接运行一次 ncu (例如使用 --set default)，其输出的 "Section: Compute Workload Analysis" 部分会列出所有捕获到的内核名称，下面也会讲Compute Workload Analysis。您也可以使用正则表达式匹配多个kernel<br>
<br>
最终输出的报告专注于你筛选出的内核。例如，如果你使用 -k "vectorAdd"，那么报告里你只会看到名为 vectorAdd 的内核的详细性能分析；如果你的程序多次启动了同一个内核，Nsight Compute 默认可能会捕获多次启动的数据。最终的报告通常会汇总这些多次执行的数据（例如平均值），或者在时间线中展示多次调用。

### 1.2.4 输出与控制

<b>ncu --csv -o report.csv ./application</b><br>
<br>
使用 ncu --csv -o report.csv ./application 命令后，得到的 CSV 文件内容会直接反映 Nsight Compute 的性能分析结果。<br>
各列通常包括：<br>
内核标识信息：例如 ID，Process ID，Process Name，Kernel Name，Context ID 等，用于区分不同的内核和其启动实例。<br>
性能指标数据：核心的性能指标列，其名称直接来自您使用 --metrics 指定的指标，或所选 --set 规则集包含的指标。例如 sm__cycles_active.avg.pct (SM 活跃周期百分比)，dram__bytes.sum.per_second (显存带宽)，l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum (全局加载请求数) 等。<br>
其他元数据：可能包括 Grid Size，Block Size，Registers Per Thread，Static Shared Memory 等内核配置信息。<br>

### 1.2.5 如何使用ncu运行某个python文件？

<b>ncu -o my_profile python your_script.py</b><br>
以上是基本用法，若需要其他更多的参数进行profile工作，可类比上述参数添加更多的分析

## 1.3 界面介绍

![界面总览](./images/view.png)

上图为界面的总览图，只需要将在服务器上生成的ncu文件下载后，拖动到NVIDIA Nsight Compute界面上，即可获知详细的profile报告

### 1.3.1 Summary

![Summary概览图](./images/summary.png)

序号 0-8 则是依次运算的kernel，其信息包括：<br>
ID: 每个函数的唯一标识符。<br>
Estimated Speedup: 估计的加速比，表示如果优化这个函数可能带来的速度提升。<br>
Function Name: 函数的名称。<br>
Demangled Name: 去掉修饰符的函数名称。<br>
Duration: 函数执行时间（以ns为单位）。<br>
Runtime Improvement: 估计的运行时间提示（以ns为单位），表示如果优化这个函数可能带来的运行时间提升。<br>
Compute Throughput: 计算吞吐量。SM 吞吐量假设在 SMSPs 间负载平衡理想的情况下 （此吞吐量指标表示在所有子单元实例的经过周期内达到的峰值持续率的百分比）。<br>
Memory Throughput: 内存吞吐量。计算内存管道吞吐量 （此吞吐量指标表示在所有子单元实例的经过周期内达到的峰值持续率的百分比）。<br>
Registers: 每个线程使用的寄存器数量。<br>
GridSize：kernel启动的网格大小。<br>
BlockSize：每个Block的线程数。<br>
Cycles：GPC指令周期。GPC：通用处理集群（General Processing Cluster）包含以 TPC（纹理处理集群）形式存在的 SM、纹理和 L1 缓存。 它在芯片上被多次复制。<br>
最上部的Result默认显示的是ID=0的 kernel 运行的部分信息，包括GPU型号及频率<br>
并且，从该图可以初步得到一些信息，比如大概率memory-bound ID8，大概率compute-bound ID7

### 1.3.2 GPU Speed Of Light Throughput

![throughput-1](./images/throughput-1.png)

左侧指标可以详细看到 Compute 和 不同层次的 Memory 的实际利用效率的情况，由此可以定位其在 roofline 中的位置。<br>
从这个结果可以看出：<br>
内存吞吐量(40.27%)高于计算吞吐量(32.70%)，表明这可能是一个内存密集型任务。<br>
L2 缓存和 DRAM 吞吐量相对较低，可能存在优化空间。<br>
L1吞吐量与总体内存吞吐量相近，说明主要的内存操作与该部分交互，需要特别说明的是 Shared memory 也统计在内。<br>
<br>
右侧指标说明了执行时间、总的周期数、SM活跃的周期数以及SM和DRAM的频率

![throughput-2](./images/throughput-2.png)

![throughput-3](./images/throughput-3.png)

最后这部分是roofiline model的一个绘制，绿色的点在斜线下面，可以很直观地判断出该kernel是memory bound的一个内核

### 1.3.3 Memory Workload Analysis

![memory-1](./images/memory-1.png)

该指标为内存资源的使用情况，主要包括通信带宽、内存指令的最大吞吐量。详细的数据表如上：<br>
<br>
Memory Throughput: 34.63 Gbyte/s<br>
即每秒在DRAM中访问的字节数。<br>
<br>
L1/TEX Hit Rate: 74.01%<br>
每个 sector 的 sector 命中次数 （这个比率指标表示跨所有子单元实例的值，以百分比表示）。<br>
<br>
l1tex：一级（L1）/纹理缓存位于GPC内部。 它可以用作定向映射的共享内存和/或在其缓存部分存储全局、本地和纹理数据。<br>
<br>
sector：缓存线或设备内存中对齐的32字节内存块。 一个L1或L2缓存线是四个sector，即128字节。 如果标签存在且sector数据在缓存线内，则sector访问被归类为命中。 标签未命中和标签命中但数据未命中都被归类为未命中。<br>
<br>
L2 Hit Rate: 87.19%<br>
L2sector查找命中的比例 （这个比率指标表示跨所有子单元实例的值，以百分比表示）。<br>
<br>
l2s：二级（L2）缓存切片是二级缓存的一个子分区。 l2s_t 指的是其标签阶段。 l2s_m 指的是其未命中阶段。 l2s_d 指的是其数据阶段。<br>
<br>
Mem Busy: 40.27%<br>
缓存和DRAM内部活动的吞吐量（这个吞吐量指标表示在所有子单元实例的经过周期内达到的峰值持续速率的百分比）<br>
<br>
Max Bandwidth: 17.53%<br>
SM<->缓存<->DRAM之间互连的吞吐量 （这个吞吐量指标表示在所有子单元实例的经过周期内达到的峰值持续速率的百分比）<br>
<br>
L2 Compression Ratio: 0
L2 Compression Success Rate: 0

![memory-2](./images/memory-2.png)

上图显示了各级 memory 的连接关系及使用情况，整体情况一目了然。<br>

![memory-3](./images/memory-3.png)

在上图中，我们主要关心load和store的情况

![memory-4](./images/memory-4.png)

在上图中，我们也主要关心load和store的情况

### 1.3.4 Compute Workload Analysis

![compute-1](./images/compute-1.png)

分析完了内存的情况后，接下来分析计算单元的使用情况。即对流式多处理器（SM）的计算资源进行详细分析，包括实际达到的每时钟周期指令数（IPC）以及每个可用流水线的利用率。主要指标包括：<br>
<br>
Executed Ipc Elapsed: 1.30 inst/cycle<br>
执行的warp指令数，此计数器指标表示所有子单元实例中每个执行周期的平均操作数。<br>
<br>
Executed Ipc Active: 1.72 inst/cycle<br>
执行的warp指令数，此计数器指标表示所有子单元实例中每个活动周期的平均操作数。<br>
<br>
Issued Ipc Active: 1.73 inst/cycle<br>
发出的warp指令数，此计数器指标表示所有子单元实例中每个活动周期的平均操作数。与上一项比较可知，在活动周期内发出的指令都被执行。<br>
<br>
SM Busy<br>
假设SMSP间理想负载平衡的SM核心指令吞吐量，此吞吐量指标表示在所有子单元实例的活动周期内达到的峰值持续率的百分比。<br>
<br>
SMSPs: 每个SM被划分为四个处理块，称为SM子分区。 SM子分区是SM上的主要处理元素。 一个子分区管理固定大小的warp池。<br>
<br>
Issue Slots Busy<br>
发出的warp指令数，此计数器指标表示在所有子单元实例的活动周期内达到的峰值持续率的平均百分比

![compute-2](./images/compute-2.png)

接下来是一些主要计算单元的利用率情况，先分别介绍一下这些计算单元：<br>

FMA: Fused Multiply Add/Accumulate，融合乘加。FMA流水线处理大多数FP 32算法（FADD、FMUL、FMAD）。它还执行整数乘法运算（IMUL、IMAD）以及整数点积。<br>
ALU: Arithmetic Logic Unit, 算术逻辑单元。ALU负责执行大多数位操作和逻辑指令。它还执行整数指令，不包括IMAD和IMUL。在NVIDIA Ampere架构芯片上，ALU流水线执行快速的FP 32到FP 16转换。<br>
LSU: Load Store Unit, 加载存储单元。LSU流水线向L1 TEX单元发出用于全局、本地和共享内存的加载、存储、原子和归约指令。它还向L1 TEX单元发出特殊的寄存器读取（S2 R）、混洗和CTA级到达/等待屏障指令。<br>
TMA: Tensor Memory Access Unit, 张量存储器访问单元。在全局内存和共享内存之间提供高效的数据传输机制，能够理解和遍历多维数据布局。<br>
ADU: Address Divergence Unit, 地址分支单元。ADU负责分支/跳转的地址发散处理。它还支持恒定加载和块级屏障指令。<br>
CBU：Convergence Barrier Unit，汇聚屏障单元。CBU负责曲速级收敛、屏障和分支指令。<br>
TEX: Texture Unit, 纹理单元。SM纹理流水线将纹理和表面指令转发到L1TEX单元的TEXIN阶段。在FP64或Tensor流水线解耦的GPU上，纹理流水线也会转发这些类型的指令。<br>
Uniform: Uniform Data Path, 统一数据路径。这个标量单元执行所有线程使用相同输入并生成相同输出的指令。<br>
XU: Transcendental and Data Type Conversion Unit, 超越和数据类型转换单元。XU管道负责特殊函数，如sin、cos和倒数平方根。它还负责int到float和float到int类型的转换.<br>

### 1.3.5 Statistics

![statistic-1](./images/statistic-1.png)

分析所有 warp 在内核执行期间所花费的周期数。warp state 描述 warp 是否准备好发出下一个指令。每条指令的 warp 周期定义了两条连续指令之间的延迟。该值越高，隐藏此延迟所需的 warp 并行度就越高。对于每个warp state，该图表显示了每个发出的指令在该状态下花费的平均周期数。stall 并不总是影响整体性能，也不是完全可以避免的。<br>
<br>
平均而言，该内核的每个warp在等待微调度器选择要发出的warp时会花费18.57个周期。未被选中的warp是指在该周期内没有被调度器选择发出的符合条件的warp。大量未被选中的warp通常意味着有足够多的warps来覆盖warp延迟，并且可以考虑减少活跃warps数量以可能增加缓存一致性和数据局部性。<br>

![statistic-2](./images/statistic-2.png)

![statistic-3](./images/statistic-3.png)

### 1.3.6 Occupancy

占用率(Occupancy)是指每个SM上活跃线程组(warp)的数量与可能的最大活跃线程组数量的比率。另一种看待占用率的方式是,它表示硬件处理线程组的能力中实际被使用的百分比。虽然较高的占用率并不总能带来更高的性能,但是低占用率会降低隐藏延迟的能力,从而导致整体性能下降。在执行过程中,理论占用率和实际达到的占用率之间存在较大差异,通常表示工作负载高度不均衡。占用率反映了GPU资源的利用情况,是评估CUDA程序性能的一个关键指标。过低的占用率会导致性能下降,需要分析并优化造成低占用率的原因。

![occupancy-1](./images/occupancy-1.png)

![occupancy-2](./images/occupancy-2.png)

## 1.4 程序示例

所有示例程序可见目录./nvidia_nsight/nsight_compute，您可以自行选择需要运行的程序，并使用ncu命令运行，然后进行一些简单的profile工作。

## 1.5 参考文献

https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#roofline<br>
https://zhuanlan.zhihu.com/p/715022552<br>
https://github.com/ifromeast/cuda_learning/blob/main/05_cuda_mode/ncu_profile/readme.md


# 2. Pytorch Profiler

## 2.1 介绍

PyTorch Profiler 是一个性能分析工具，专门用于分析和优化 PyTorch 模型的训练和推理性能。它提供了：<br>
时间分析：测量操作执行时间<br>
内存分析：跟踪内存分配和使用情况<br>
GPU 利用率：分析 GPU 内核执行效率<br>
可视化：通过 TensorBoard 提供直观的可视化界面<br>

## 2.2 常用指令

### 2.2.1 基础配置参数

In [ ]:
from torch.profiler import profile, ProfilerActivity, tensorboard_trace_handler

# 基础配置示例
profiler_config = {
    "activities": [
        ProfilerActivity.CPU,      # 分析 CPU 活动
        ProfilerActivity.CUDA,     # 分析 GPU 活动
    ],
    "schedule": torch.profiler.schedule(
        skip_first=3,    # 跳过前3次step
        wait=1,          # 等待1次step
        warmup=1,        # 预热1次step
        active=3,        # 活跃记录3次step
        repeat=2         # 重复2轮
    ),
    "on_trace_ready": tensorboard_trace_handler("./logs"),
    "record_shapes": True,        # 记录张量形状
    "profile_memory": True,       # 分析内存使用
    "with_stack": True,           # 记录调用栈
    "with_flops": True,           # 计算 FLOPs
    "with_modules": True,         # 按模块分组
}

### 2.2.2 核心类和方法

torch.profiler.profile

In [ ]:
# 完整的 profiler 配置
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    
    # 调度配置
    schedule=torch.profiler.schedule(
        skip_first=5,
        wait=2,
        warmup=2,
        active=5,
        repeat=1
    ),
    
    # 输出配置
    on_trace_ready=tensorboard_trace_handler("./logs"),
    
    # 数据记录配置
    record_shapes=True,      # 记录输入形状
    profile_memory=True,     # 内存分析
    with_stack=True,         # 调用栈信息
    with_flops=True,         # FLOPs 计算
    with_modules=False,      # 模块级分析
    
    # 实验性功能
    experimental_config=torch.profiler._ExperimentalConfig(
        profiler_metrics=torch.profiler.ProfilerMetrics.CUDA_LAUNCH_STATS
    )
) as prof:
    # 训练循环
    for step, data in enumerate(dataloader):
        # 模型训练代码
        train_step(data)
        
        # 重要：在每个step后调用
        prof.step()
        
        if step >= 20:  # 限制分析步数
            break

调度器 (schedule)

In [1]:
def create_profiler_schedule():
    """创建不同的调度策略"""
    
    # 方案1: 训练初期分析
    early_training = schedule(
        skip_first=0,   # 不跳过
        wait=0,         # 不等待
        warmup=1,       # 预热1步
        active=5,       # 记录5步
        repeat=1        # 只执行1轮
    )
    
    # 方案2: 稳定期分析
    stable_training = schedule(
        skip_first=50,  # 跳过前50步
        wait=5,         # 等待5步
        warmup=2,       # 预热2步
        active=10,      # 记录10步
        repeat=2        # 执行2轮
    )
    
    # 方案3: 周期性分析
    periodic_analysis = schedule(
        skip_first=10,
        wait=5,
        warmup=2,
        active=3,
        repeat=5        # 执行5轮，形成周期性分析
    )
    
    return early_training

### 2.2.3 分析方法

基础分析

In [1]:
def basic_analysis(prof):
    """基础性能分析"""
    
    print("=== 按CUDA时间排序 ===")
    print(prof.key_averages().table(
        sort_by="cuda_time_total", 
        row_limit=15
    ))
    
    print("\n=== 按CPU时间排序 ===")
    print(prof.key_averages().table(
        sort_by="cpu_time_total", 
        row_limit=15
    ))
    
    print("\n=== 按内存使用排序 ===")
    print(prof.key_averages().table(
        sort_by="self_cpu_memory_usage", 
        row_limit=10
    ))

高级分析

In [2]:
def advanced_analysis(prof):
    """高级分析技巧"""
    
    # 1. 按操作类型分组
    print("=== 按操作类型分组 ===")
    key_averages = prof.key_averages()
    print(key_averages.table(
        sort_by="cuda_time_total",
        row_limit=10  # 可选：限制显示的行数
    ))

    # 2. 按输入形状分组
    print("\n=== 按输入形状分组 ===")
    key_averages_by_input_shape = prof.key_averages(group_by_input_shape=True)
    print(key_averages_by_input_shape.table(
        sort_by="cpu_time_total",
        row_limit=10  # 可选：限制显示的行数
    ))
    
    # 3. 总统计信息
    print("\n=== 总体统计 ===")
    total_stats = prof.key_averages().total_average()
    print(f"总CPU时间: {total_stats.cpu_time_total / 1000:.2f} ms")
    print(f"总CUDA时间: {total_stats.cuda_time_total / 1000:.2f} ms")
    
    # 4. 事件统计
    events = prof.events()
    print(f"\n总事件数: {len(events)}")

内存分析

In [3]:
def memory_analysis(prof):
    """内存使用分析"""
    
    print("=== 内存分析 ===")
    
    # CPU 内存分析
    print("\n--- CPU内存使用 ---")
    cpu_memory_table = prof.key_averages().table(
        sort_by="self_cpu_memory_usage",
        row_limit=10
    )
    print(cpu_memory_table)
    
    # CUDA 内存分析
    print("\n--- CUDA内存使用 ---")
    cuda_memory_table = prof.key_averages().table(
        sort_by="self_cuda_memory_usage", 
        row_limit=10
    )
    print(cuda_memory_table)
    
    # 内存时间线
    print("\n=== 内存事件 ===")
    for event in prof.events():
        if hasattr(event, 'cpu_memory_usage') and event.cpu_memory_usage > 0:
            print(f"操作: {event.name}, CPU内存: {event.cpu_memory_usage} bytes")

### 2.2.4 实用工具函数

性能报告生成器

In [4]:
def generate_performance_report(prof, model_name="model"):
    """生成详细的性能报告"""
    
    report = f"""
=== {model_name} 性能分析报告 ===

执行统计:
{prof.key_averages().table(sort_by="cuda_time_total", row_limit=20)}

内存分析:
{prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=10)}

CPU密集型操作:
{prof.key_averages().table(sort_by="cpu_time_total", row_limit=10)}

关键指标:
"""
    
    # 计算关键指标
    total_stats = prof.key_averages().total_average()
    cpu_time_ms = total_stats.cpu_time_total / 1000
    cuda_time_ms = total_stats.cuda_time_total / 1000
    
    report += f"""
- 总CPU时间: {cpu_time_ms:.2f} ms
- 总CUDA时间: {cuda_time_ms:.2f} ms
- CPU-CUDA时间比: {cpu_time_ms/cuda_time_ms:.2f}
"""
    
    return report

瓶颈检测

In [5]:
def detect_bottlenecks(prof, threshold_ms=10.0):
    """检测性能瓶颈"""
    
    print("=== 性能瓶颈检测 ===")
    
    key_averages = prof.key_averages()
    bottlenecks = []
    
    for event in key_averages:
        cuda_time_ms = event.cuda_time_total / 1000
        cpu_time_ms = event.cpu_time_total / 1000
        
        if cuda_time_ms > threshold_ms or cpu_time_ms > threshold_ms:
            bottlenecks.append({
                'name': event.key,
                'cuda_time_ms': cuda_time_ms,
                'cpu_time_ms': cpu_time_ms,
                'input_shape': str(event.input_shapes) if hasattr(event, 'input_shapes') else 'N/A'
            })
    
    # 打印瓶颈
    for i, bottleneck in enumerate(sorted(bottlenecks, key=lambda x: x['cuda_time_ms'], reverse=True)):
        print(f"{i+1}. {bottleneck['name']}")
        print(f"   CUDA时间: {bottleneck['cuda_time_ms']:.2f} ms")
        print(f"   CPU时间: {bottleneck['cpu_time_ms']:.2f} ms")
        print(f"   输入形状: {bottleneck['input_shape']}")
        print()
    
    return bottlenecks

## 2.3 界面介绍

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.profiler import profile, schedule, tensorboard_trace_handler, ProfilerActivity

class ComprehensiveProfilerExample:
    def __init__(self):
        self.model = self.create_model()
        self.optimizer = optim.Adam(self.model.parameters())
        self.criterion = nn.CrossEntropyLoss()
        
    def create_model(self):
        return nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 10)
        ).cuda()
    
    def run_profiling(self):
        """运行完整的性能分析"""
        
        # 配置profiler
        prof = profile(
            activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
            schedule=schedule(
                skip_first=2,
                wait=1,
                warmup=1,
                active=3,
                repeat=2
            ),
            #on_trace_ready=tensorboard_trace_handler("./log"),
            record_shapes=True,
            profile_memory=True,
            with_stack=True,
            with_flops=True
        )
        
        prof.start()
        
        for step in range(20):
            # 模拟训练步骤
            inputs = torch.randn(32, 3, 32, 32).cuda()
            targets = torch.randint(0, 10, (32,)).cuda()
            
            # 前向传播
            outputs = self.model(inputs)
            loss = self.criterion(outputs, targets)
            
            # 反向传播
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            # 推进profiler
            prof.step()
            
        prof.stop()
        
        # 分析结果
        self.analyze_results(prof)
        
    def analyze_results(self, prof):
        """分析profiler结果"""
        
        print("=" * 60)
        print("PyTorch Profiler 分析结果")
        print("=" * 60)
        
        # 1. 基础分析
        basic_analysis(prof)
        
        # 2. 高级分析
        advanced_analysis(prof)
        
        # 3. 内存分析
        memory_analysis(prof)
        
        # 4. 生成报告
        report = generate_performance_report(prof, "CNN_Model")
        print(report)
        
        # 5. 瓶颈检测
        bottlenecks = detect_bottlenecks(prof, threshold_ms=5.0)

        # 6. 保存跟踪文件
        prof.export_chrome_trace("comprehensive_trace.json")
        print("跟踪文件已保存: comprehensive_trace.json")

if __name__ == "__main__":
    example = ComprehensiveProfilerExample()
    example.run_profiling()

/home/hangcheng.dong/miniconda3/envs/bevfusion/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch Profiler 分析结果


NameError: name 'basic_analysis' is not defined

以上是界面介绍的示例函数

## 2.4 程序示例

## 2.5 参考文献

# 3. NVIDIA Nsight System

## 3.1 介绍

NVIDIA Nsight Systems（nsys）作为NVIDIA官方提供的系统级性能分析工具，能够深入到硬件执行层面，提供从CPU线程调度到GPU kernel执行的完整时间线视图，帮助开发者：<br>
精确定位性能瓶颈：无论是计算受限、内存带宽受限，还是CPU-GPU同步开销过大<br>
量化优化效果：通过详细的性能指标对比，验证优化策略的实际效果<br>
理解系统行为：深入理解现代GPU应用程序的执行机制和资源利用模式<br>
在大模型训练、推理服务、科学计算等关键应用场景中，nsys已经成为性能工程师不可或缺的利器。掌握nsys的使用方法和分析技巧，对于任何从事GPU性能优化工作的开发者都具有重要的实用价值。<br>

## 3.2 常用指令

### 3.2.1 总体流程


1. 平台支持<br>
Nsight Systems（nsys）是跨平台工具，支持 Windows、Linux、macOS，提供对应的 CLI 和 GUI 版本。<br>
2. 数据采集<br>
使用 nsys CLI 工具在目标环境（服务器 / 多 GPU 系统）上运行程序，收集性能数据。<br>
生成 .nsys-rep 或 .qdrep 文件（请求文件 / trace 文件）。<br>
3. 数据分析<br>
将生成的 trace 文件拷贝到个人 PC 或分析工作站。<br>
使用 nsys GUI 工具加载文件，可视化分析 GPU 时间线、CPU-GPU 交互、内存传输、Kernel 执行、库函数性能等。<br>
可进行筛选、放大关键阶段、对比不同运行配置等操作。<br>

### 3.2.2 采集模式

nsys 提供两种性能抓取方式：<br>
1. 直接 Profile 模式<br>
在命令行中预先指定要抓取的内容，以及起止条件（如持续时间、直到程序结束）。<br>
工具会在达到条件时自动停止抓取。<br>
优点：简单易用，适合全流程或确定阶段的性能采集；无需人工干预。<br>
缺点：缺乏灵活性，无法根据程序实际运行情况临时调整抓取时机，容易产生无关数据或错过关键片段。<br>
2. 交互式 Profile 模式<br>
通过 nsys launch 启动目标进程，在另一终端中手动触发开始/结束。<br>
可以在观察到程序运行状态后，再选择具体时段进行抓取。<br>
同时交互式可以抓取多次。<br>
优点：更灵活，能针对性捕捉不同阶段的性能数据，减少无效数据采集，文件体积也更小。<br>
缺点：需要人工干预，操作复杂度更高，不适合无人值守或批量任务。<br>
如果明确知道要分析的阶段，且希望流程自动化，推荐使用 直接 profile；<br>
如果想在程序运行过程中观察状态，并灵活抓取关键时段，则 交互式 profile 更合适。<br>
⚠️无论用哪种模式，收集时间尽量控制在1min以内，否则太大无法分析。<br>

#### Profile模式

profile模式：启动并分析新进程<br>  
nsys profile torchrun train.py<br>
参数说明：用torchrun代指实际的训练启动命令。<br>
当前启动之后，nsys 会在train.py 进程结束之后生成一个nsys-rep后缀的文件。

#### Lauch模式

<b>1. 先启动任务</b><br>
nsys launch torchrun train.py<br>
<b>2. 新开一个窗口</b><br>
<b>可以看到 sessions正在运行</b><br>
nsys status<br>
<b>3. 启动、结束</b><br>
nsys start<br>
nsys stop<br>

#### 关键参数配置详解

详细参数可见，有很多开关，这里仅仅常用收集参数。<br>
https://docs.nvidia.com/nsight-systems/UserGuide/index.html<br>

一般来说，使用如下命令收集：<br>

nsys profile <br>
  --trace cuda,osrt,nvtx,cudnn,cublas  <br>
  --gpu-metrics-device=all <br>
  --duration=60 <br>
  --delay=120 <br>
  --cuda-memory-usage true <br>
  --output profile_log <br>
    torchrun train.py <br>
    
    
 # 参数说明：
   --trace cuda,osrt,nvtx,cudnn,cublas # 收集的内容，默认为cuda opengl nvtx osrt<br>
   --gpu-metrics-device=all  # gpu相关指标<br>
   --duration=30 \ #收集时间为30s<br>
   --delay=120 \ # 延迟120s进行收集<br>
   --cuda-memory-usage true \ # 显存使用情况<br>
     --output profile_log \ # 指定输出路径和文件名<br>

注意事项
1. 延迟视情况而定，比如如果训练在30分钟之后拉起，需要延迟3600s
2. 收集时间不宜过长，控制在60s之内
3. 默认输入路径为当前路径，建议更改到有权限的输出路径，避免写到/tmp导致保存失败
4. 输出路径多节点共享存储，输出文件名要有hostname之类的区别，否则会被覆盖写入保存失败
5. 假设平台调度，存在某一个pod写完其他pod会被杀死的情况，建议使用launch方式收集
6. ⚠️注意CPU权限配置，如果非管理员权限采集CPU详细信息，需要修改系统配置
7. echo 0 | sudo tee /proc/sys/kernel/perf_event_paranoid

### 3.2.3 NVTX标记技巧

NVTX（NVIDIA Tools Extension） 是 NVIDIA 提供的一套标注工具接口。<br>
<br>
它允许开发者在代码中插入 标记（marker） 或 区间（range），用来标识某段逻辑或操作。<br>
<br>
在使用 nsys 进行性能分析时，这些标记会显示在时间轴上，方便开发者快速定位和分析程序的关键阶段<br>
<br>
如下图所示，可以清晰的看到程序每一步运行所在的状态。<br>

#### python中NVTX的使用

pip install nvtx 即可使用：<br>
主要使用方式：<br>

In [ ]:
import nvtx

@nvtx.annotate("data_loading", color="blue")
def load_data():
    # 数据加载逻辑
    pass

# 上下文管理器
with nvtx.annotate("backward_pass", color="red"):
    loss.backward()
  
# 手动标记1
range_id = nvtx.start_range("my_code_range", domain="my_domain")
loss.backward()
nvtx.end_range(range_id)

# 手动标记2
import time
import nvtx
nvtx.push_range("my_code_range", domain="my_domain")
model.train()
nvtx.pop_range(domain="my_domain")

新版的nvtx还提供了两种自动注解函数名字的方式：


In [ ]:
# 直接在代码中使用nvtx.Profile()
import nvtx
import time
pr = nvtx.Profile()
pr.enable()
time.sleep(1)
pr.disable()
time.sleep(1)

# 将nvtx作为命令行工具调用
# 功能不用改源码，自动给所有函数加注释
python -m nvtx script.py


同时新版nsys可以对pytroch自动加函数nvtx了，非常方便。<br>
 --pytorch=autograd-nvtx  可以自动增加pytorch 函数的 nvtx

## 3.3 界面介绍

### 引言：CPU-GPU协同的编程范式

在深度学习训练或推理中，CPU 与 GPU 协同执行是性能分析的基础。理解这一协同机制对于优化深度学习系统性能至关重要。通常流程可以拆解为以下几个层次：

#### 应用层（Python/Pytorch）

<b>用户代码层面</b>：大多数深度学习逻辑写在 Python 层（如 nn.Module、loss.backward()），这层代码主要负责模型定义、训练循环控制和数据流管理。<br>

<b>框架调度层面</b>：PyTorch 作为中间层，封装了底层算子调用，通过 C++ ATen 库和 CUDA Runtime 将高级操作转换为具体的 GPU kernel 调用。每个 PyTorch 操作（如 torch.matmul、torch.relu）会被分解为一个或多个 CUDA kernel。<br>

<b>关键理解</b>：<br>

Python 代码本身永远不在 GPU 上运行<br>
每次张量操作都会触发 kernel launch 开销，频繁的小操作会导致性能损失<br>
算子融合（operator fusion）可以减少 kernel 启动次数，提升整体性<br>

#### Host 与 Device 的内存模型

在 GPU 编程中，Host 一般代指 CPU 及其相关资源，Device 一般代指 GPU 及其相关资源。理解两者的内存特性和数据传输机制是性能优化的关键。

内存架构对比<br>

Host Memory（CPU 内存，DDR）：<br>
数据源头：Dataloader 首先将批次数据从存储设备加载到系统内存<br>
代码执行：存储模型代码、Python 解释器、深度学习框架逻辑等运行时环境<br>
容量特点：容量大（通常 64-512GB），成本相对较低<br>
性能特点：访问延迟较高（~100-300ns），带宽相对有限（~100GB/s）<br>
适用场景：数据预处理、存储、控制逻辑执行<br>
Device Memory（GPU 显存，HBM）：<br>
计算要求：GPU 所有计算必须在显存中完成，这是 GPU 架构的硬性要求<br>
容量限制：容量相对有限（通常 16-80GB），成本较高，是深度学习模型规模的重要约束<br>
性能优势：超高带宽（~4TB/s），低延迟访问，专为并行计算优化<br>
存储内容：模型参数、输入数据、中间激活值、梯度、优化器状态等训练/推理所需的所有数据<br>

#### 数据传输路径

现代深度学习系统中的数据传输主要包含三种模式：<br>

H2D (Host-to-Device) 传输：<br>
传输内容：训练数据、模型参数、配置信息从 CPU 内存传输到 GPU 显存<br>
典型场景：批次数据上传、模型权重初始化、超参数传递<br>
性能瓶颈：PCIe 带宽限制（~32GB/s），通常是训练流水线的主要瓶颈<br>
优化策略：Pinned Memory、异步传输、数据预取<br>

D2H (Device-to-Host) 传输：<br>
传输内容：训练结果、损失值、模型检查点从 GPU 显存传输到 CPU 内存<br>
典型场景：损失值回传、模型保存、中间结果验证、调试信息提取<br>
频率特点：通常频率较低，但对调试和监控至关重要<br>
注意事项：会触发 GPU-CPU 同步，可能打断训练流水线<br>

D2D (Device-to-Device) 传输：<br>
传输内容：GPU 间直接数据交换，bypass CPU 和系统内存<br>
典型场景：<br>
分布式训练：梯度同步（AllReduce）、参数广播（Broadcast）<br>
模型并行：不同 GPU 负责模型的不同部分，需要交换中间激活<br>
数据并行：多 GPU 间的负载均衡和结果聚合<br>
技术实现：<br>
NVLink：NVIDIA GPU 间的专用高速互联（~600GB/s），NVSwitch支持多 GPU 全连接的交换矩阵<br>
RDMA：跨节点的高速网络互联<br>
性能优势：带宽远高于 PCIe，延迟更低，是大规模分布式训练的基础<br>

#### Stream 并发与 CPU↔GPU 协作

##### CUDA Stream 机制

Stream 概念：CUDA Stream 是 GPU 上的任务执行队列，保证队列内任务按顺序执行：<br>
CPU 将任务（kernel 启动、内存拷贝、同步操作）提交到指定 Stream<br>
Stream 内串行，Stream 间并行：不同 Stream 上的任务可以并发执行<br>
默认使用 "default stream"，也可创建多个用户 Stream

##### 执行模式对比

异步模式（默认推荐）：

In [ ]:
# CPU 提交任务后立即返回，不等待 GPU 完成
x_gpu = x.cuda()          # 异步内存拷贝
y = torch.matmul(x_gpu, w)  # 异步 kernel 启动
z = torch.relu(y)         # 异步 kernel 启动

优势：<br>
CPU 将任务提交到 Stream 后立即返回，可以并行准备下一个 batch<br>
GPU 在后台执行，CPU 与 GPU 时间重叠<br>
数据传输和计算可以在不同 Stream 上并发，提升整体利用率<br>

同步模式（阻塞调用）：<br>
export CUDA_LAUNCH_BLOCKING=1  # 环境变量强制同步<br>
<br>
特点：<br>
强制 CPU 等待每个 GPU 操作完成<br>
CPU 与 GPU 都失去并行性，性能显著下降<br>
主要用于调试定位错误（便于确定具体哪个 kernel 出错）<br>
由于默认异步执行，CPU 可能在 GPU 尚未完成计算时就继续执行后续代码，导致：<br>
数据竞争：访问尚未计算完成的结果<br>
时序错误：依赖关系被破坏<br>
错误的性能测量：计时不准确<br>
<br>


常用同步方式<br>
PyTorch 同步 API：

In [ ]:

torch.cuda.synchronize()        # 等待所有 Stream 完成
torch.cuda.synchronize(device)  # 等待指定设备完成

CUDA 原生同步：

In [ ]:
import pycuda.driver as cuda
cuda.Context.synchronize()      # 等待当前 context 完成
stream.synchronize()            # 等待特定 stream 完成

隐式同步场景：<br>
数据回传：.cpu() 或 .numpy() 会触发隐式同步<br>
跨设备操作：不同 GPU 间的数据拷贝<br>
Host 端访问：tensor.item() 获取标量值<br>
<br>
典型应用场景<br>
精确计时 Benchmark：<br>


In [ ]:
torch.cuda.synchronize()
start_time = time.time()
result = model(input_data)
torch.cuda.synchronize()  # 确保计算完成
end_time = time.time()
#调试错误定位：
# 每个操作后同步，精确定位出错位置
x = torch.matmul(a, b)
torch.cuda.synchronize()  # 如果上一行出错，这里会抛出异常

#### GPU 计算模型

##### Kernel 执行机制

Kernel Launch 过程：<br>
CPU 通过 CUDA Runtime API（如 cudaLaunchKernel）启动 kernel<br>
配置执行参数：grid 维度、block 维度、shared memory 大小等<br>
异步提交到 GPU 执行队列，CPU 可立即返回继续执行<br>

##### 大规模并行架构

线程层次结构：<br>
Thread：最小执行单元，每个线程处理一个数据元素<br>
Warp：32个线程为一组，SIMD 执行（单指令多数据）<br>
Block：多个 warp 组成，共享 shared memory 和同步原语<br>
Grid：多个 block 组成，对应一次 kernel 启动<br>
调度特点：<br>
GPU 拥有数千个 CUDA core，可同时执行数万个线程<br>
warp 级调度：当一个 warp 等待内存访问时，调度器切换到其他warp<br>
通过大量线程掩盖访存延迟，实现高吞吐量计算<br>

##### 内存层次结构

#### 常见性能瓶颈与优化策略

### 3.3.1 GPU Timeline 完整视图

主要内容：CUDA API 调用时序- GPU Kernel 执行时间线- Memory Transfer 可视化- Stream 并发情况<br>
价值：展示 GPU 上的执行全貌，帮助定位 Kernel 重叠度、数据传输与计算的重叠情况

### 3.3.2 CPU-GPU 交互分析

主要内容：Host–Device 数据传输瓶颈识别- CPU 等待 GPU 的空闲时间- 异步调用效率分析<br>
价值：揭示 CPU 与 GPU 协同是否高效，发现数据传输或同步点带来的性能损耗

### 3.3.3 多层次性能指标

主要内容：GPU 利用率统计- Memory 带宽利用率- Kernel Launch 开销- Context Switch 分析<br>
价值：提供量化指标，帮助分析算力使用是否饱和、是否存在频繁小 Kernel 等低效模式

### 3.3.4 系统资源监控

主要内容：GPU 温度 / 功耗- PCIe 带宽使用情况- 多 GPU 系统的负载均衡<br>
价值：从硬件维度辅助性能调优，避免功耗过高、PCIe 拥塞或 GPU 负载不均衡

### 3.3.5 第三方库透明分析

主要内容：cuDNN、cuBLAS 等库性能- 自定义 CUDA kernel 性能- 驱动层面的性能瓶颈<br>
价值：无需修改库源码即可分析性能，定位到算子级 / kernel 级瓶颈，区分是库问题还是自定义代码问题

## 3.4 程序示例

## 3.5 参考文献